<a href="https://colab.research.google.com/github/AndreiAf02/Job_Satisfaction_Burnout/blob/main/Job_Satisfaction_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import math
from scipy import stats
from scipy.stats import mannwhitneyu

In [2]:
def bootstrap_analysis(bootstrap_path_values, Working_condition_cols):
  mean_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  lbound_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  ubound_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  pbci_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  pbci_90_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)

  for path in bootstrap_path_values['Path'].unique().tolist():
    data = bootstrap_path_values[bootstrap_path_values['Path']==path].reset_index(drop=True)
    mean_vals = [path]
    lbound_vals = [path]
    ubound_vals = [path]
    pbci_vals = [path]
    pbci_90_vals = [path]

    for col in Working_condition_cols:
      mean_vals.append(data[col].mean())
      ## Calculating the lower and upper bounds of the Percentile Bootstrap Confidence Interval (PBCI):
      lbound = np.percentile(data[col], 2.5)
      ubound = np.percentile(data[col], 97.5)
      if lbound > 0. or ubound < 0.:
        sig = ' **'
      elif np.percentile(data[col], 5) > 0 or np.percentile(data[col], 95) < 0:
        sig = ' *'
      else:
        sig = ''
      lbound_vals.append(lbound)
      ubound_vals.append(ubound)
      pbci_vals.append('['+str(round(lbound,4))+', '+str(round(ubound,4))+']'+sig)
      pbci_90_vals.append('['+str(round(np.percentile(data[col], 5.),4))+', '+str(round(np.percentile(data[col], 95),4))+']')

    mean_results.loc[len(mean_results)] = mean_vals
    lbound_results.loc[len(lbound_results)] = lbound_vals
    ubound_results.loc[len(ubound_results)] = ubound_vals
    pbci_results.loc[len(pbci_results)] = pbci_vals
    pbci_90_results.loc[len(pbci_90_results)] = pbci_90_vals



  return mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results

In [3]:
def Bayesian_analysis(result_posterior_samples, factor_unique, dir_path_unique):
  path_ci = pd.DataFrame()
  path_prob = pd.DataFrame()

  path_ci['Factor'] = factor_unique
  path_prob['Factor'] = factor_unique

  med_ci_vals = []
  med_prob_vals = []
  mod_ci_vals = []
  mod_prob_vals = []
  dp_ci_vals = []
  dp_prob_vals = []

  for i in range(len(factor_unique)):

    col = factor_unique[i]
    ## Finding the Mediation Credibility Intervals and probabilities:
    data = result_posterior_samples[f'Indirect effect ({col})']
    lbound = np.percentile(data, 2.5)
    ubound = np.percentile(data, 97.5)
    probability_of_positivity = np.mean(data > 0)
    if lbound >0 or ubound < 0:
      sig = ' **'
    elif probability_of_positivity < 0.05 or probability_of_positivity > 0.95:
      sig = ' *'
    else:
      sig = ''
    med_ci_vals.append(f'[{lbound:.4f}, {ubound:.4f}]'+sig)
    med_prob_vals.append(probability_of_positivity)


    ## Finding the Moderation Credibility Intervals and probabilities:
    data_mod = result_posterior_samples[f'Moderation effect ({col})']
    lbound_mod = np.percentile(data_mod, 2.5)
    ubound_mod = np.percentile(data_mod, 97.5)
    probability_of_positivity_mod = np.mean(data_mod > 0)
    if lbound_mod >0 or ubound_mod < 0:
      sig_mod = ' **'
    elif probability_of_positivity_mod < 0.05 or probability_of_positivity_mod > 0.95:
      sig_mod = ' *'
    else:
      sig_mod = ''
    mod_ci_vals.append(f'[{lbound_mod:.4f}, {ubound_mod:.4f}]'+sig_mod)
    mod_prob_vals.append(probability_of_positivity_mod)


    ## Finding the Direct Pathway Credibility Intervals and probabilities:
    data_dp = result_posterior_samples[dir_path_unique[i]]
    lbound_dp = np.percentile(data_dp, 2.5)
    ubound_dp = np.percentile(data_dp, 97.5)
    probability_of_positivity_dp = np.mean(data_dp > 0)
    if lbound_dp >0 or ubound_dp < 0:
      sig_dp = ' **'
    elif probability_of_positivity_dp < 0.05 or probability_of_positivity_dp > 0.95:
      sig_dp = ' *'
    else:
      sig_dp = ''
    dp_ci_vals.append(f'[{lbound_dp:.4f}, {ubound_dp:.4f}]'+sig_dp)
    dp_prob_vals.append(probability_of_positivity_dp)


  path_ci['Mediation'] = med_ci_vals
  path_prob['Mediation'] = med_prob_vals
  path_ci['Moderation'] = mod_ci_vals
  path_prob['Moderation'] = mod_prob_vals
  path_ci['Direct Path'] = dp_ci_vals
  path_prob['Direct Path'] = dp_prob_vals

  return path_ci, path_prob

In [4]:
def Bayesian_analysis_no_mod(result_posterior_samples, factor_unique, dir_path_unique):
  path_ci = pd.DataFrame()
  path_prob = pd.DataFrame()

  path_ci['Factor'] = factor_unique
  path_prob['Factor'] = factor_unique

  med_ci_vals = []
  med_prob_vals = []
  mod_ci_vals = []
  mod_prob_vals = []
  dp_ci_vals = []
  dp_prob_vals = []

  for i in range(len(factor_unique)):

    col = factor_unique[i]
    ## Finding the Mediation Credibility Intervals and probabilities:
    data = result_posterior_samples[f'Indirect effect ({col})']
    lbound = np.percentile(data, 2.5)
    ubound = np.percentile(data, 97.5)
    probability_of_positivity = np.mean(data > 0)
    if lbound >0 or ubound < 0:
      sig = ' **'
    elif probability_of_positivity < 0.05 or probability_of_positivity > 0.95:
      sig = ' *'
    else:
      sig = ''
    med_ci_vals.append(f'[{lbound:.5f}, {ubound:.5f}]'+sig)
    med_prob_vals.append(probability_of_positivity)


    ## Finding the Direct Pathway Credibility Intervals and probabilities:
    data_dp = result_posterior_samples[dir_path_unique[i]]
    lbound_dp = np.percentile(data_dp, 2.5)
    ubound_dp = np.percentile(data_dp, 97.5)
    probability_of_positivity_dp = np.mean(data_dp > 0)
    if lbound_dp >0 or ubound_dp < 0:
      sig_dp = ' **'
    elif probability_of_positivity_dp < 0.05 or probability_of_positivity_dp > 0.95:
      sig_dp = ' *'
    else:
      sig_dp = ''
    dp_ci_vals.append(f'[{lbound_dp:.5f}, {ubound_dp:.5f}]'+sig_dp)
    dp_prob_vals.append(probability_of_positivity_dp)


  path_ci['Mediation'] = med_ci_vals
  path_prob['Mediation'] = med_prob_vals
  path_ci['Direct Path'] = dp_ci_vals
  path_prob['Direct Path'] = dp_prob_vals

  return path_ci, path_prob

In [5]:
def Bayesian_analysis_multi_group_comparison(result_posterior_samples1, result_posterior_samples2, factor_unique, dir_path_unique, post1, post2, start_pos):

  for i in range(len(factor_unique)):
    col = factor_unique[i]
    ## Finding the Mediation Credibility Intervals and probabilities:
    data1 = result_posterior_samples1[f'Indirect effect ({col})']
    data2 = result_posterior_samples2[f'Indirect effect ({col})']
    data_diff = data2 - data1
    diff_mean = data_diff.mean()
    lbound = np.percentile(data_diff, 2.5)
    ubound = np.percentile(data_diff, 97.5)
    probability_of_positivity = np.mean(data_diff > 0)
    if lbound >0 or ubound < 0:
      sig = ' **'
    elif probability_of_positivity < 0.05 or probability_of_positivity > 0.95:
      sig = ' *'
    else:
      sig = ''
    if probability_of_positivity < 0.05 or probability_of_positivity > 0.95:
      if probability_of_positivity < 0.05:
        probability_of_positivity = probability_of_positivity
      elif probability_of_positivity > 0.95:
        probability_of_positivity = 1 - probability_of_positivity
      (f'Indirect Effect ({col}) - Group {post1+start_pos} vs. {post2+start_pos}: prob = {probability_of_positivity:.4f}, mean = {diff_mean:.4f}, CrI = [{lbound:.4f}, {ubound:.4f}]'+sig)



    ## Finding the Moderation Credibility Intervals and probabilities:
    data_mod1 = result_posterior_samples1[f'Moderation effect ({col})']
    data_mod2 = result_posterior_samples2[f'Moderation effect ({col})']
    data_diff_mod = data_mod2 - data_mod1
    diff_mean_mod = data_diff_mod.mean()
    lbound_mod = np.percentile(data_diff_mod, 2.5)
    ubound_mod = np.percentile(data_diff_mod, 97.5)
    probability_of_positivity_mod = np.mean(data_diff_mod > 0)
    if lbound_mod >0 or ubound_mod < 0:
      sig_mod = ' **'
    elif probability_of_positivity_mod < 0.05 or probability_of_positivity_mod > 0.95:
      sig_mod = ' *'
    else:
      sig_mod = ''
    if probability_of_positivity_mod < 0.05 or probability_of_positivity_mod > 0.95:
      if probability_of_positivity_mod < 0.05:
        probability_of_positivity_mod = probability_of_positivity_mod
      elif probability_of_positivity_mod > 0.95:
        probability_of_positivity_mod = 1 - probability_of_positivity_mod
      print(f'Moderation Effect ({col}) - Group {post1+start_pos} vs. {post2+start_pos}: prob = {probability_of_positivity_mod:.4f}, mean = {diff_mean_mod:.4f}, CrI = [{lbound_mod:.4f}, {ubound_mod:.4f}]'+sig_mod)


    ## Finding the Direct Pathway Credibility Intervals and probabilities:
    data_dp1 = result_posterior_samples1[dir_path_unique[i]]
    data_dp2 = result_posterior_samples2[dir_path_unique[i]]
    data_diff_dp = data_dp2 - data_dp1
    diff_mean_dp = data_diff_dp.mean()
    lbound_dp = np.percentile(data_diff_dp, 2.5)
    ubound_dp = np.percentile(data_diff_dp, 97.5)
    probability_of_positivity_dp = np.mean(data_diff_dp > 0)
    if lbound_dp >0 or ubound_dp < 0:
      sig_dp = ' **'
    elif probability_of_positivity_dp < 0.05 or probability_of_positivity_dp > 0.95:
      sig_dp = ' *'
    else:
      sig_dp = ''
    if probability_of_positivity_dp < 0.05 or probability_of_positivity_dp > 0.95:
      if probability_of_positivity_dp < 0.05:
        probability_of_positivity_dp = probability_of_positivity_dp
      elif probability_of_positivity_dp > 0.95:
        probability_of_positivity_dp = 1 - probability_of_positivity_dp
      print(f'Direct Path ({col}) - Group {post1+start_pos} vs {post2+start_pos}: prob = {probability_of_positivity_dp:.4f}, mean = {diff_mean_dp:.4f}, CrI = [{lbound_dp:.4f}, {ubound_dp:.4f}]'+sig_dp)
      print('-'*50)

In [6]:
def bootstrap_analysis_single_group(bootstrap_path_values):
  mean_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  lbound_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  ubound_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  pbci_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  pbci_90_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)

  for path in bootstrap_path_values['Path'].unique().tolist():
    data = bootstrap_path_values[bootstrap_path_values['Path']==path].reset_index(drop=True)
    mean_vals = [path]
    lbound_vals = [path]
    ubound_vals = [path]
    pbci_vals = [path]
    pbci_90_vals = [path]

    for col in Working_condition_cols:
      mean_vals.append(data[col].mean())
      ## Calculating the lower and upper bounds of the Percentile Bootstrap Confidence Interval (PBCI):
      lbound = np.percentile(data[col], 2.5)
      ubound = np.percentile(data[col], 97.5)
      if lbound > 0. or ubound < 0.:
        sig = ' **'
      elif np.percentile(data[col], 5) > 0 or np.percentile(data[col], 95) < 0:
        sig = ' *'
      else:
        sig = ''
      lbound_vals.append(lbound)
      ubound_vals.append(ubound)
      pbci_vals.append('['+str(round(lbound,4))+', '+str(round(ubound,4))+']'+sig)
      pbci_90_vals.append('['+str(round(np.percentile(data[col], 5.),4))+', '+str(round(np.percentile(data[col], 95),4))+']')

    mean_results.loc[len(mean_results)] = mean_vals
    lbound_results.loc[len(lbound_results)] = lbound_vals
    ubound_results.loc[len(ubound_results)] = ubound_vals
    pbci_results.loc[len(pbci_results)] = pbci_vals
    pbci_90_results.loc[len(pbci_90_results)] = pbci_90_vals

  return mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results


In [7]:
def bayesian_analysis_single_group(result_posterior_samples, Current_position_col):
  factor_unique = ['Salary', 'Effort', 'Teaching',
              'Research', 'Policy', 'Acad Resources', 'Autonomy']

  dir_path_unique = ['dp1', 'dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7']

  dp_mod_unique = ['dp_mod1', 'dp_mod2', 'dp_mod3', 'dp_mod4', 'dp_mod5', 'dp_mod6', 'dp_mod7']

  path_ci = pd.DataFrame()
  path_prob = pd.DataFrame()

  path_ci['Factor'] = factor_unique
  path_prob['Factor'] = factor_unique

  med_ci_vals = []
  med_prob_vals = []
  dp_ci_vals = []
  dp_prob_vals = []

  ## Calculating the position moderation effects on burnout:
  for i in range(len(Current_position_col)):
    mod_ci_vals = []
    mod_prob_vals = []
    for j in range(len(factor_unique)):
      col = factor_unique[j]
      ## Finding the Moderation Credibility Intervals and probabilities:
      group = [f'Moderation effect ({col}) - Group '+str(i+1)]
      data_mod = result_posterior_samples[group]
      lbound_mod = np.percentile(data_mod, 2.5)
      ubound_mod = np.percentile(data_mod, 97.5)
      probability_of_positivity_mod = np.mean(data_mod > 0)
      if lbound_mod >0 or ubound_mod < 0:
        sig_mod = ' **'
      elif probability_of_positivity_mod <= 0.05 or probability_of_positivity_mod >= 0.95:
        sig_mod = ' *'
      else:
        sig_mod = ''
      mod_ci_vals.append(f'[{lbound_mod:.4f}, {ubound_mod:.4f}]'+sig_mod)
      mod_prob_vals.append(probability_of_positivity_mod)
    path_ci['Moderation - Group '+str(i+1)] = mod_ci_vals
    path_prob['Moderation - Group '+str(i+1)] = mod_prob_vals

  ## Calculating the Moderation effects on the direct path to job satisfaction:
  for i in range(len(Current_position_col)):
    dp_mod_ci_vals = []
    dp_mod_prob_vals = []
    for dp_mod in dp_mod_unique:
      group_dp_mod = [dp_mod+str(int(i+1))]
      data_dp_mod = result_posterior_samples[group_dp_mod]
      lbound_dp_mod = np.percentile(data_dp_mod, 2.5)
      ubound_dp_mod = np.percentile(data_dp_mod, 97.5)
      probability_of_positivity_dp_mod = np.mean(data_dp_mod > 0)
      if lbound_dp_mod >0 or ubound_dp_mod < 0:
        sig_dp_mod = ' **'
      elif probability_of_positivity_dp_mod <= 0.05 or probability_of_positivity_dp_mod >= 0.95:
        sig_dp_mod = ' *'
      else:
        sig_dp_mod = ''
      dp_mod_ci_vals.append(f'[{lbound_dp_mod:.4f}, {ubound_dp_mod:.4f}]'+sig_dp_mod)
      dp_mod_prob_vals.append(probability_of_positivity_dp_mod)
    path_ci['Direct Path Moderation - Group '+str(i+1)] = dp_mod_ci_vals
    path_prob['Direct Path Moderation - Group '+str(i+1)] = dp_mod_prob_vals



  for i in range(len(factor_unique)):
    col = factor_unique[i]
    ## Finding the Mediation Credibility Intervals and probabilities:
    data = result_posterior_samples[f'Indirect effect ({col})']
    lbound = np.percentile(data, 2.5)
    ubound = np.percentile(data, 97.5)
    probability_of_positivity = np.mean(data > 0)
    if lbound >0 or ubound < 0:
      sig = ' **'
    elif probability_of_positivity <= 0.05 or probability_of_positivity >= 0.95:
      sig = ' *'
    else:
      sig = ''
    med_ci_vals.append(f'[{lbound:.4f}, {ubound:.4f}]'+sig)
    med_prob_vals.append(probability_of_positivity)


    ## Finding the Direct Pathway Credibility Intervals and probabilities:
    data_dp = result_posterior_samples[dir_path_unique[i]]
    lbound_dp = np.percentile(data_dp, 2.5)
    ubound_dp = np.percentile(data_dp, 97.5)
    probability_of_positivity_dp = np.mean(data_dp > 0)
    if lbound_dp >0 or ubound_dp < 0:
      sig_dp = ' **'
    elif probability_of_positivity_dp <= 0.05 or probability_of_positivity_dp >= 0.95:
      sig_dp = " *"
    else:
      sig_dp = ''
    dp_ci_vals.append(f'[{lbound_dp:.4f}, {ubound_dp:.4f}]'+sig_dp)
    dp_prob_vals.append(probability_of_positivity_dp)


  path_ci['Mediation'] = med_ci_vals
  path_prob['Mediation'] = med_prob_vals
  path_ci['Direct Path'] = dp_ci_vals
  path_prob['Direct Path'] = dp_prob_vals

  return path_ci, path_prob

## Czech Republic:
**Bootstrap:**

In [8]:
path_to_bootstrap = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bootstrap/CZ/'

## Path values with no moderation:
bootstrap_vals_no_mod_pos_3 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_3['Position'] = 3.
bootstrap_vals_no_mod_pos_4 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_4['Position'] = 4.
bootstrap_vals_no_mod_pos_5 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_5['Position'] = 5.
bootstrap_vals_no_mod_pos_6 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_6['Position'] = 6.

## Path values with moderation:
bootstrap_vals_pos_3 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_3['Position'] = 3.
bootstrap_vals_pos_4 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_4['Position'] = 4.
bootstrap_vals_pos_5 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_5['Position'] = 5.
bootstrap_vals_pos_6 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_6.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_6['Position'] = 6.




In [9]:
bootstrap_vals_no_mod_pos_3

,Bootstrap_nr,Path,Salary,Effort,Policy_influence,Teaching_percent,Research_percent,Academic_Resources,Perceived_Autonomy,DoF,...,CFI,GFI,AGFI,NFI,TLI,RMSEA,AIC,BIC,LogLik,Position
0,0,Mediation,-0.001130,-0.000797,0.004235,-0.000562,-0.000899,0.020858,-0.000439,28.0,...,1.151834,1.000000,1.000000,1.000000,1.233174,0.000000,33.999999,72.703323,2.755053e-07,3.0
1,0,Direct Path,-0.006148,0.005930,-0.052816,-0.002966,-0.006086,0.311291,0.397314,28.0,...,1.151834,1.000000,1.000000,1.000000,1.233174,0.000000,33.999999,72.703323,2.755053e-07,3.0
2,1,Mediation,0.006756,-0.001507,-0.038026,0.001365,0.001380,0.027286,0.027807,28.0,...,0.800997,0.723859,0.575926,0.723859,0.694389,0.144660,32.066617,70.769941,9.666917e-01,3.0
3,1,Direct Path,0.026954,0.003077,0.194853,-0.003903,-0.009183,0.387597,0.313352,28.0,...,0.800997,0.723859,0.575926,0.723859,0.694389,0.144660,32.066617,70.769941,9.666917e-01,3.0
4,2,Mediation,-0.008592,-0.001652,-0.015295,-0.000224,-0.000874,0.008115,0.038858,28.0,...,0.859687,0.765613,0.640049,0.765613,0.784520,0.115955,32.479723,71.183047,7.601383e-01,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1997,Direct Path,0.057162,0.002172,-0.152243,-0.000192,-0.007716,0.320129,0.350899,28.0,...,0.751420,0.681317,0.510594,0.681317,0.618253,0.159688,31.814047,70.517371,1.092977e+00,3.0
3996,1998,Mediation,-0.000400,-0.000284,-0.007577,0.000056,0.000058,0.002692,0.006318,28.0,...,0.647326,0.579745,0.354609,0.579745,0.458393,0.161444,31.782906,70.486230,1.108547e+00,3.0
3997,1998,Direct Path,0.068411,0.000415,0.006469,-0.001855,-0.001824,0.345606,0.036366,28.0,...,0.647326,0.579745,0.354609,0.579745,0.458393,0.161444,31.782906,70.486230,1.108547e+00,3.0
3998,1999,Mediation,-0.002692,-0.000750,-0.005051,0.000466,-0.000226,0.026953,0.015448,28.0,...,0.675054,0.617450,0.412512,0.617450,0.500975,0.181037,31.412344,70.115668,1.293828e+00,3.0


In [10]:
Working_condition_cols = ['Salary', 'Effort', 'Policy_influence', 'Teaching_percent',
                          'Research_percent', 'Academic_Resources', 'Perceived_Autonomy']


In [11]:
## Analyzing the bootstrap CI for the paths with age moderation:
i = 2

# Mediation_results = pd.DataFrame()
# Moderation_results = pd.DataFrame()
# Direct_path_results = pd.DataFrame()
for bootstrap_vals in [bootstrap_vals_pos_3, bootstrap_vals_pos_4, bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals, Working_condition_cols)
  pbci_results_transposed = pbci_results.set_index('Path').T
  # Mediation_results['Position '+str(i)] = pbci_results_transposed['Mediation']
  # Moderation_results['Position '+str(i)] = pbci_results_transposed['Moderation']
  # Direct_path_results['Position '+str(i)] = pbci_results_transposed['Direct Path']
  print(pbci_results_transposed)
  print('-'*50)

# Mediation_results

95% CI for Position 3:
Path                        Mediation         Moderation          Direct Path
Salary              [-0.0159, 0.0359]  [-0.0012, 0.0028]    [-0.0267, 0.1135]
Effort              [-0.0071, 0.0028]  [-0.0004, 0.0001]    [-0.0017, 0.0113]
Policy_influence    [-0.0505, 0.0549]  [-0.0033, 0.0054]    [-0.2053, 0.1001]
Teaching_percent    [-0.0028, 0.0055]  [-0.0002, 0.0004]      [-0.01, 0.0079]
Research_percent    [-0.0039, 0.0039]  [-0.0003, 0.0003]    [-0.0118, 0.0055]
Academic_Resources  [-0.0597, 0.0955]  [-0.0044, 0.0084]    [0.1287, 0.52] **
Perceived_Autonomy  [-0.0908, 0.0994]   [-0.009, 0.0049]  [0.1041, 0.5497] **
--------------------------------------------------
95% CI for Position 4:
Path                            Mediation         Moderation  \
Salary                  [-0.0035, 0.0051]  [-0.0007, 0.0003]   
Effort              [-0.0015, -0.0002] **   [-0.0, 0.0001] *   
Policy_influence     [-0.0246, -0.003] **  [-0.0013, 0.0012]   
Teaching_percent       

In [12]:
for factor in bootstrap_vals_pos_3.columns[2:9]:
  print('++'*40)
  print(factor)
  for bootstrap_vals1 in [bootstrap_vals_pos_3, bootstrap_vals_pos_4,
                        bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
    for bootstrap_vals2 in [bootstrap_vals_pos_3, bootstrap_vals_pos_4,
                        bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
      position1 = bootstrap_vals1['Position'].unique()
      position2 = bootstrap_vals2['Position'].unique()
      if position1[0] < position2[0]:
        for path in ['Mediation', 'Moderation', 'Direct Path']:
          # print(f'Group 1: Position {position1[0]}')
          # print(f'Group 2: Position {position2[0]}')
          data1 = bootstrap_vals1[factor][bootstrap_vals1['Path']==path].sort_values().reset_index(drop=True)
          data2 = bootstrap_vals2[factor][bootstrap_vals2['Path']==path].sort_values().reset_index(drop=True)
          data_diff = bootstrap_vals2[factor][bootstrap_vals2['Path']==path] - bootstrap_vals1[factor][bootstrap_vals1['Path']==path]
          data_diff = data_diff.sort_values().reset_index(drop=True)
          diff_mean = data_diff.mean()
          lbound = np.percentile(data_diff, 2.5)
          ubound = np.percentile(data_diff, 97.5)
          if lbound > 0 or ubound < 0:
            sig = ' **'
          elif np.percentile(data_diff, 5) > 0 or np.percentile(data_diff, 95) < 0:
            sig = ' *'
          else:
            sig  = ''
          # print(f'Difference PBCI for Position {position1[0]} vs {position2[0]} - {path} ({factor}): [{lbound:.4f}, {ubound:.4f}]{sig}')
          if np.percentile(data_diff, 5) > 0 or np.percentile(data_diff, 95) < 0:
            # print(f'Group 1: Position {position1[0]}')
            # print(f'Group 2: Position {position2[0]}')
            print(f'Difference PBCI for Position {position1[0]} vs {position2[0]} - {path} ({factor}): mean = {diff_mean:.4f}, CI = [{lbound:.4f}, {ubound:.4f}]{sig}')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Salary
Difference PBCI for Position 4.0 vs 5.0 - Direct Path (Salary): mean = -0.0371, CI = [-0.0738, -0.0008] **
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Effort
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Policy_influence
Difference PBCI for Position 3.0 vs 6.0 - Direct Path (Policy_influence): mean = 0.2297, CI = [0.0008, 0.4395] **
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Teaching_percent
Difference PBCI for Position 3.0 vs 6.0 - Direct Path (Teaching_percent): mean = 0.0138, CI = [-0.0022, 0.0319] *
Difference PBCI for Position 4.0 vs 6.0 - Direct Path (Teaching_percent): mean = 0.0144, CI = [0.0003, 0.0309] **
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Research_percent
Difference PBCI for Position 4.0 vs 5.0 - Moderation (Research_percent): mean = -

In [13]:
## Analyzing the bootstrap CI for the paths without age moderation:
# Mediation_results_no_mod = pd.DataFrame()
# Direct_path_results_no_mod = pd.DataFrame()
i = 2
for bootstrap_vals_no_mod in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                       bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals_no_mod, Working_condition_cols)
  pbci_results_transposed = pbci_results.set_index('Path').T
  # Mediation_results_no_mod['Position '+str(i)] = pbci_results_transposed['Mediation']
  # Direct_path_results_no_mod['Position '+str(i)] = pbci_results_transposed['Direct Path']
  print(pbci_results_transposed)
  print('-'*50)

# Mediation_results_no_mod


95% CI for Position 3:
Path                        Mediation          Direct Path
Salary              [-0.0071, 0.0135]    [-0.0267, 0.1135]
Effort              [-0.0028, 0.0011]    [-0.0017, 0.0113]
Policy_influence     [-0.042, 0.0184]    [-0.2054, 0.1001]
Teaching_percent     [-0.001, 0.0016]      [-0.01, 0.0079]
Research_percent     [-0.0018, 0.001]    [-0.0118, 0.0055]
Academic_Resources  [-0.0253, 0.0522]  [0.1288, 0.5197] **
Perceived_Autonomy  [-0.0225, 0.0695]   [0.104, 0.5497] **
--------------------------------------------------
95% CI for Position 4:
Path                            Mediation          Direct Path
Salary                  [-0.0023, 0.0062]  [0.0357, 0.0732] **
Effort              [-0.0018, -0.0004] **     [-0.0023, 0.002]
Policy_influence     [-0.0227, -0.002] **  [0.0203, 0.1042] **
Teaching_percent        [-0.0011, 0.0006]    [-0.0051, 0.0018]
Research_percent        [-0.0009, 0.0012]  [-0.0074, 0.0002] *
Academic_Resources      [-0.0062, 0.0303]  [0.4627, 0

In [14]:
for factor in bootstrap_vals_no_mod_pos_3.columns[2:9]:
  print('++'*40)
  print(factor)
  for bootstrap_vals1 in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                        bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
    for bootstrap_vals2 in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                        bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
      position1 = bootstrap_vals1['Position'].unique()
      position2 = bootstrap_vals2['Position'].unique()
      if position1[0] < position2[0]:
        for path in ['Mediation', 'Direct Path']:
          # print(f'Group 1: Position {position1[0]}')
          # print(f'Group 2: Position {position2[0]}')
          data1 = bootstrap_vals1[factor][bootstrap_vals1['Path']==path].sort_values().reset_index(drop=True)
          data2 = bootstrap_vals2[factor][bootstrap_vals2['Path']==path].sort_values().reset_index(drop=True)
          data_diff = bootstrap_vals2[factor][bootstrap_vals2['Path']==path] - bootstrap_vals1[factor][bootstrap_vals1['Path']==path]
          data_diff = data_diff.sort_values().reset_index(drop=True)

          lbound = np.percentile(data_diff, 2.5)
          ubound = np.percentile(data_diff, 97.5)
          if lbound > 0 or ubound < 0:
            sig = ' **'
          elif np.percentile(data_diff, 5) > 0 or np.percentile(data_diff, 95) < 0:
            sig = ' *'
          else:
            sig  = ''
          # print(f'Difference PBCI for Position {position1[0]} vs {position2[0]} - {path} ({factor}): [{lbound:.4f}, {ubound:.4f}]{sig}')
          if np.percentile(data_diff, 5) > 0 or np.percentile(data_diff, 95) < 0:
            # print(f'Group 1: Position {position1[0]}')
            # print(f'Group 2: Position {position2[0]}')
            print(f'Difference PBCI for Position {position1[0]} vs {position2[0]} - {path} ({factor}): [{lbound:.4f}, {ubound:.4f}]{sig}')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Salary
Difference PBCI for Position 4.0 vs 5.0 - Direct Path (Salary): [-0.0738, -0.0007] **
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Effort
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Policy_influence
Difference PBCI for Position 3.0 vs 6.0 - Direct Path (Policy_influence): [0.0008, 0.4395] **
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Teaching_percent
Difference PBCI for Position 3.0 vs 6.0 - Direct Path (Teaching_percent): [-0.0022, 0.0319] *
Difference PBCI for Position 4.0 vs 6.0 - Direct Path (Teaching_percent): [0.0003, 0.0309] **
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Research_percent
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Academic_Resources
Difference PBCI for Position 3.0 vs 4.0 - Direct Path (Academi

**Bayesian:**

In [15]:
path_to_bayesian = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bayesian/CZ/'

## Path values without moderation:
bayesian_vals_no_mod_pos_3 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_4 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_5 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_6 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]

## Path values with moderation:
bayesian_vals_pos_3 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_4 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_5 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_6 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_6.0.csv', sep=",", header=0).iloc[:,1:]

In [16]:
factor_unique = ['Salary', 'Effort', 'Teaching',
              'Research', 'Policy', 'Acad Resources', 'Autonomy']

dir_path_unique = ['dp1', 'dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7']


In [17]:
i = 2
for bayesian_vals in [bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis(bayesian_vals, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 3:
                        Mediation         Moderation          Direct Path
Factor                                                                   
Salary          [-0.0130, 0.0321]  [-0.0009, 0.0024]  [-0.0088, 0.1095] *
Effort          [-0.0074, 0.0034]  [-0.0004, 0.0002]    [-0.0028, 0.0116]
Teaching        [-0.0019, 0.0048]  [-0.0001, 0.0003]    [-0.0101, 0.0079]
Research        [-0.0028, 0.0029]  [-0.0002, 0.0002]    [-0.0140, 0.0062]
Policy          [-0.0315, 0.0481]  [-0.0026, 0.0052]    [-0.2079, 0.0997]
Acad Resources  [-0.0398, 0.0984]  [-0.0030, 0.0072]  [0.1151, 0.5397] **
Autonomy        [-0.0622, 0.0528]  [-0.0073, 0.0029]  [0.1529, 0.5453] **
--------------------------------------------------
95% CrI for Position 4:
                            Mediation           Moderation  \
Factor                                                       
Salary              [-0.0043, 0.0051]    [-0.0008, 0.0002]   
Effort          [-0.0014, -0.0002] **  [0.0000, 0

In [18]:
bayesian_posterior_list = [bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]

for post1 in range(len(bayesian_posterior_list)):
  for post2 in range(len(bayesian_posterior_list)):
    if post1 < post2:
      # print('i:', i, 'j:', j)
      result_posterior_samples1 = bayesian_posterior_list[post1]
      result_posterior_samples2 = bayesian_posterior_list[post2]
      Bayesian_analysis_multi_group_comparison(result_posterior_samples1, result_posterior_samples2, factor_unique, dir_path_unique, post1, post2, 3)

Direct Path (Acad Resources) - Group 3 vs 4: prob = 0.0365, mean = 0.2128, CrI = [-0.0192, 0.4403] *
--------------------------------------------------
Direct Path (Policy) - Group 3 vs 5: prob = 0.0475, mean = 0.1461, CrI = [-0.0269, 0.3253] *
--------------------------------------------------
Direct Path (Acad Resources) - Group 3 vs 5: prob = 0.0195, mean = 0.2981, CrI = [0.0312, 0.5649] **
--------------------------------------------------
Direct Path (Policy) - Group 3 vs 6: prob = 0.0185, mean = 0.2382, CrI = [0.0275, 0.4538] **
--------------------------------------------------
Direct Path (Salary) - Group 4 vs 5: prob = 0.0210, mean = -0.0372, CrI = [-0.0712, -0.0019] **
--------------------------------------------------
Moderation Effect (Research) - Group 4 vs. 5: prob = 0.0245, mean = -0.0002, CrI = [-0.0005, -0.0000] **
Direct Path (Teaching) - Group 4 vs 6: prob = 0.0285, mean = 0.0134, CrI = [-0.0004, 0.0273] *
--------------------------------------------------


In [19]:
i = 2
for bayesian_vals_no_mod in [bayesian_vals_no_mod_pos_3, bayesian_vals_no_mod_pos_4,
                             bayesian_vals_no_mod_pos_5, bayesian_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis_no_mod(bayesian_vals_no_mod, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 3:
                          Mediation            Direct Path
Factor                                                    
Salary          [-0.00828, 0.01575]  [-0.01237, 0.11104] *
Effort          [-0.00337, 0.00136]    [-0.00267, 0.01235]
Teaching        [-0.00129, 0.00218]    [-0.01010, 0.00767]
Research        [-0.00233, 0.00173]    [-0.01249, 0.00582]
Policy          [-0.04645, 0.01901]    [-0.20690, 0.09658]
Acad Resources  [-0.03334, 0.04298]  [0.09797, 0.53165] **
Autonomy        [-0.02519, 0.07364]  [0.13963, 0.54675] **
--------------------------------------------------
95% CrI for Position 4:
                              Mediation            Direct Path
Factor                                                        
Salary              [-0.00244, 0.00607]  [0.03740, 0.07102] **
Effort          [-0.00174, -0.00043] **    [-0.00231, 0.00207]
Teaching            [-0.00114, 0.00052]    [-0.00506, 0.00165]
Research            [-0.00084, 0.00109]  [-0.00742, 0.0

## Austria:
**Bootstrap:**

In [20]:
path_to_bootstrap = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bootstrap/AT/'

## Path values with no moderation:
bootstrap_vals_no_mod_pos_2 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_2['Position'] = 2.
bootstrap_vals_no_mod_pos_3 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_3['Position'] = 3.
bootstrap_vals_no_mod_pos_4 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_4['Position'] = 4.
bootstrap_vals_no_mod_pos_5 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_5['Position'] = 5.
bootstrap_vals_no_mod_pos_6 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_6['Position'] = 6.

## Path values with moderation:
bootstrap_vals_pos_2 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_2['Position'] = 2.
bootstrap_vals_pos_3 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_3['Position'] = 3.
bootstrap_vals_pos_4 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_4['Position'] = 4.
bootstrap_vals_pos_5 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_5['Position'] = 5.
bootstrap_vals_pos_6 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_6.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_6['Position'] = 6.




In [21]:
Working_condition_cols = ['Salary', 'Effort', 'Policy_influence', 'Teaching_percent',
                          'Research_percent', 'Academic_Resources', 'Perceived_Autonomy']


In [22]:
## Analyzing the bootstrap CI for the paths with age moderation:
i = 1
for bootstrap_vals in [bootstrap_vals_pos_2, bootstrap_vals_pos_3, bootstrap_vals_pos_4, bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals, Working_condition_cols)
  print(pbci_results.set_index('Path').T)
  print('-'*50)


95% CI for Position 2:
Path                        Mediation         Moderation  \
Salary              [-0.0207, 0.0329]   [-0.001, 0.0007]   
Effort              [-0.0046, 0.0023]  [-0.0001, 0.0001]   
Policy_influence    [-0.1555, 0.0582]  [-0.0019, 0.0054]   
Teaching_percent    [-0.0111, 0.0041]  [-0.0001, 0.0004]   
Research_percent    [-0.0065, 0.0039]  [-0.0001, 0.0002]   
Academic_Resources  [-0.1172, 0.1577]   [-0.005, 0.0039]   
Perceived_Autonomy  [-0.0481, 0.2348]  [-0.0064, 0.0015]   

Path                          Direct Path  
Salary                   [-0.015, 0.0145]  
Effort              [-0.0076, -0.0001] **  
Policy_influence         [-0.025, 0.1887]  
Teaching_percent        [-0.0031, 0.0098]  
Research_percent      [-0.0001, 0.0087] *  
Academic_Resources    [0.2012, 0.4553] **  
Perceived_Autonomy    [0.2805, 0.5122] **  
--------------------------------------------------
95% CI for Position 3:
Path                        Mediation         Moderation          Dire

In [23]:
for factor in bootstrap_vals_pos_3.columns[2:9]:
  print('++'*40)
  print(factor)
  for bootstrap_vals1 in [bootstrap_vals_pos_2, bootstrap_vals_pos_3, bootstrap_vals_pos_4,
                        bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
    for bootstrap_vals2 in [bootstrap_vals_pos_2, bootstrap_vals_pos_3, bootstrap_vals_pos_4,
                        bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
      position1 = bootstrap_vals1['Position'].unique()
      position2 = bootstrap_vals2['Position'].unique()
      if position1[0] < position2[0]:
        for path in ['Mediation', 'Moderation', 'Direct Path']:
          # print(f'Group 1: Position {position1[0]}')
          # print(f'Group 2: Position {position2[0]}')
          data1 = bootstrap_vals1[factor][bootstrap_vals1['Path']==path].sort_values().reset_index(drop=True)
          data2 = bootstrap_vals2[factor][bootstrap_vals2['Path']==path].sort_values().reset_index(drop=True)
          data_diff = bootstrap_vals2[factor][bootstrap_vals2['Path']==path] - bootstrap_vals1[factor][bootstrap_vals1['Path']==path]
          data_diff = data_diff.sort_values().reset_index(drop=True)
          diff_mean = data_diff.mean()

          lbound = np.percentile(data_diff, 2.5)
          ubound = np.percentile(data_diff, 97.5)
          if lbound > 0 or ubound < 0:
            sig = ' **'
          elif np.percentile(data_diff, 5) > 0 or np.percentile(data_diff, 95) < 0:
            sig = ' *'
          else:
            sig  = ''
          # print(f'Difference PBCI for Position {position1[0]} vs {position2[0]} - {path} ({factor}): [{lbound:.4f}, {ubound:.4f}]{sig}')
          if np.percentile(data_diff, 5) > 0 or np.percentile(data_diff, 95) < 0:
            print(f'Difference PBCI for Position {position1[0]} vs {position2[0]} - {path} ({factor}): mean = {diff_mean:.4f}, CI = [{lbound:.4f}, {ubound:.4f}]{sig}')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Salary
Difference PBCI for Position 2.0 vs 5.0 - Mediation (Salary): mean = -0.1368, CI = [-0.3219, -0.0070] **
Difference PBCI for Position 2.0 vs 5.0 - Moderation (Salary): mean = 0.0026, CI = [0.0001, 0.0067] **
Difference PBCI for Position 3.0 vs 5.0 - Mediation (Salary): mean = -0.1462, CI = [-0.3333, -0.0127] **
Difference PBCI for Position 3.0 vs 5.0 - Moderation (Salary): mean = 0.0028, CI = [0.0001, 0.0068] **
Difference PBCI for Position 5.0 vs 6.0 - Mediation (Salary): mean = 0.1289, CI = [-0.0046, 0.3172] *
Difference PBCI for Position 5.0 vs 6.0 - Moderation (Salary): mean = -0.0025, CI = [-0.0064, 0.0000] *
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Effort
Difference PBCI for Position 2.0 vs 5.0 - Mediation (Effort): mean = -0.1281, CI = [-0.2993, 0.0121] *
Difference PBCI for Position 2.0 vs 5.0 - Moderation (Effort): mean = 0.0029, CI = [-0.0002, 0.0066

In [24]:
## Analyzing the bootstrap CI for the paths without age moderation:
i = 1
for bootstrap_vals_no_mod in [bootstrap_vals_no_mod_pos_2, bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                       bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals_no_mod, Working_condition_cols)
  print(pbci_results.set_index('Path').T)
  print('-'*50)

95% CI for Position 2:
Path                        Mediation            Direct Path
Salary              [-0.0018, 0.0044]       [-0.015, 0.0145]
Effort              [-0.0009, 0.0003]  [-0.0076, -0.0001] **
Policy_influence    [-0.0071, 0.0225]       [-0.025, 0.1888]
Teaching_percent    [-0.0007, 0.0009]      [-0.0031, 0.0098]
Research_percent    [-0.0005, 0.0006]    [-0.0001, 0.0087] *
Academic_Resources  [-0.0144, 0.0204]     [0.201, 0.4552] **
Perceived_Autonomy    [-0.02, 0.0547]    [0.2805, 0.5121] **
--------------------------------------------------
95% CI for Position 3:
Path                          Mediation          Direct Path
Salary              [0.0001, 0.0138] **    [-0.0027, 0.0325]
Effort                [-0.0028, 0.0003]    [-0.0062, 0.0076]
Policy_influence      [-0.0034, 0.0776]    [-0.1634, 0.1678]
Teaching_percent        [-0.002, 0.002]     [-0.006, 0.0108]
Research_percent      [-0.0006, 0.0027]    [-0.0029, 0.0107]
Academic_Resources    [-0.0055, 0.0987]  [0.0591,

In [25]:
for factor in bootstrap_vals_pos_3.columns[2:9]:
  print('++'*40)
  print(factor)
  for bootstrap_vals1 in [bootstrap_vals_no_mod_pos_2, bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                       bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
    for bootstrap_vals2 in [bootstrap_vals_no_mod_pos_2, bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                       bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
      position1 = bootstrap_vals1['Position'].unique()
      position2 = bootstrap_vals2['Position'].unique()
      if position1[0] < position2[0]:
        for path in ['Mediation', 'Direct Path']:
          # print(f'Group 1: Position {position1[0]}')
          # print(f'Group 2: Position {position2[0]}')
          data1 = bootstrap_vals1[factor][bootstrap_vals1['Path']==path].sort_values().reset_index(drop=True)
          data2 = bootstrap_vals2[factor][bootstrap_vals2['Path']==path].sort_values().reset_index(drop=True)
          data_diff = bootstrap_vals2[factor][bootstrap_vals2['Path']==path] - bootstrap_vals1[factor][bootstrap_vals1['Path']==path]
          data_diff = data_diff.sort_values().reset_index(drop=True)

          lbound = np.percentile(data_diff, 2.5)
          ubound = np.percentile(data_diff, 97.5)
          if lbound > 0 or ubound < 0:
            sig = ' **'
          elif np.percentile(data_diff, 5) > 0 or np.percentile(data_diff, 95) < 0:
            sig = ' *'
          else:
            sig  = ''
          # print(f'Difference PBCI for Position {position1[0]} vs {position2[0]} - {path} ({factor}): [{lbound:.4f}, {ubound:.4f}]{sig}')
          if np.percentile(data_diff, 5) > 0 or np.percentile(data_diff, 95) < 0:
            print(f'Difference PBCI for Position {position1[0]} vs {position2[0]} - {path} ({factor}): [{lbound:.4f}, {ubound:.4f}]{sig}')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Salary
Difference PBCI for Position 3.0 vs 6.0 - Mediation (Salary): [-0.0144, -0.0002] **
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Effort
Difference PBCI for Position 2.0 vs 5.0 - Direct Path (Effort): [-0.0264, 0.0016] *
Difference PBCI for Position 3.0 vs 5.0 - Direct Path (Effort): [-0.0316, -0.0017] **
Difference PBCI for Position 5.0 vs 6.0 - Direct Path (Effort): [0.0001, 0.0290] **
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Policy_influence
Difference PBCI for Position 3.0 vs 6.0 - Mediation (Policy_influence): [-0.0842, 0.0015] *
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Teaching_percent
Difference PBCI for Position 2.0 vs 6.0 - Direct Path (Teaching_percent): [-0.0181, 0.0000] *
Difference PBCI for Position 3.0 vs 6.0 - Direct Path (Teaching_percent): [-0.0181, 0.0014] *
Difference

**Bayesian:**

In [26]:
path_to_bayesian = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bayesian/AT/'

## Path values without moderation:
bayesian_vals_no_mod_pos_2 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_3 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_4 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_5 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_6 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]

## Path values with moderation:
bayesian_vals_pos_2 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_3 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_4 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_5 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_6 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_6.0.csv', sep=",", header=0).iloc[:,1:]

In [27]:
factor_unique = ['Salary', 'Effort', 'Teaching',
              'Research', 'Policy', 'Acad Resources', 'Autonomy']

dir_path_unique = ['dp1', 'dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7']


In [28]:
i = 1
for bayesian_vals in [bayesian_vals_pos_2, bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis(bayesian_vals, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 2:
                        Mediation         Moderation            Direct Path
Factor                                                                     
Salary          [-0.0148, 0.0389]  [-0.0011, 0.0004]      [-0.0131, 0.0188]
Effort          [-0.0057, 0.0020]  [-0.0001, 0.0002]  [-0.0077, -0.0001] **
Teaching        [-0.0103, 0.0036]  [-0.0001, 0.0004]      [-0.0035, 0.0099]
Research        [-0.0062, 0.0035]  [-0.0001, 0.0002]    [-0.0004, 0.0088] *
Policy          [-0.1387, 0.0575]  [-0.0018, 0.0049]      [-0.0181, 0.1857]
Acad Resources  [-0.0637, 0.1488]  [-0.0045, 0.0021]    [0.1991, 0.4615] **
Autonomy        [-0.0562, 0.1652]  [-0.0040, 0.0018]    [0.2793, 0.5040] **
--------------------------------------------------
95% CrI for Position 3:
                        Mediation         Moderation          Direct Path
Factor                                                                   
Salary          [-0.0118, 0.0553]  [-0.0013, 0.0005]    [-0.0050, 0.0

In [29]:
bayesian_posterior_list = [bayesian_vals_pos_2, bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]

for post1 in range(len(bayesian_posterior_list)):
  for post2 in range(len(bayesian_posterior_list)):
    if post1 < post2:
      # print('i:', i, 'j:', j)
      result_posterior_samples1 = bayesian_posterior_list[post1]
      result_posterior_samples2 = bayesian_posterior_list[post2]
      Bayesian_analysis_multi_group_comparison(result_posterior_samples1, result_posterior_samples2, factor_unique, dir_path_unique, post1, post2, 2)

Direct Path (Teaching) - Group 2 vs 6: prob = 0.0170, mean = -0.0091, CrI = [-0.0175, -0.0006] **
--------------------------------------------------
Direct Path (Research) - Group 2 vs 6: prob = 0.0245, mean = -0.0091, CrI = [-0.0176, -0.0001] **
--------------------------------------------------
Direct Path (Effort) - Group 3 vs 5: prob = 0.0170, mean = -0.0150, CrI = [-0.0289, -0.0016] **
--------------------------------------------------
Direct Path (Teaching) - Group 3 vs 6: prob = 0.0395, mean = -0.0079, CrI = [-0.0167, 0.0010] *
--------------------------------------------------
Direct Path (Research) - Group 3 vs 6: prob = 0.0405, mean = -0.0089, CrI = [-0.0183, 0.0012] *
--------------------------------------------------
Direct Path (Effort) - Group 5 vs 6: prob = 0.0280, mean = 0.0129, CrI = [-0.0003, 0.0263] *
--------------------------------------------------
Direct Path (Research) - Group 5 vs 6: prob = 0.0360, mean = -0.0213, CrI = [-0.0436, 0.0013] *
---------------------

In [30]:
i = 1
for bayesian_vals_no_mod in [bayesian_vals_no_mod_pos_2, bayesian_vals_no_mod_pos_3, bayesian_vals_no_mod_pos_4,
                             bayesian_vals_no_mod_pos_5, bayesian_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis_no_mod(bayesian_vals_no_mod, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 2:
                          Mediation              Direct Path
Factor                                                      
Salary          [-0.00142, 0.00436]      [-0.01258, 0.01935]
Effort          [-0.00097, 0.00031]  [-0.00798, -0.00028] **
Teaching        [-0.00076, 0.00080]      [-0.00271, 0.00977]
Research        [-0.00063, 0.00050]    [-0.00045, 0.00846] *
Policy          [-0.00798, 0.02187]    [-0.01520, 0.18916] *
Acad Resources  [-0.01180, 0.02161]    [0.19556, 0.45909] **
Autonomy        [-0.02402, 0.06292]    [0.27814, 0.50197] **
--------------------------------------------------
95% CrI for Position 3:
                            Mediation            Direct Path
Factor                                                      
Salary          [0.00042, 0.01377] **    [-0.00432, 0.03595]
Effort            [-0.00321, 0.00039]    [-0.00540, 0.00718]
Teaching          [-0.00207, 0.00213]    [-0.00516, 0.00944]
Research          [-0.00095, 0.00243]    [-0.00

## Single Group Model Analysis:

In [31]:
path_to_single_group = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Single_Group/'

CZ_bootstrap_single_group = pd.read_csv(path_to_single_group+'CZ_bootstrap_Single_Group.csv', sep=",", header=0).iloc[:,1:]
CZ_bayesian_single_group = pd.read_csv(path_to_single_group+'CZ_burnout_posterior_Single_Group.csv', sep=",", header=0).iloc[:,1:]
AT_bootstrap_single_group = pd.read_csv(path_to_single_group+'AT_bootstrap_Single_Group.csv', sep=",", header=0).iloc[:,1:]
AT_bayesian_single_group = pd.read_csv(path_to_single_group+'AT_burnout_posterior_Single_Group.csv', sep=",", header=0).iloc[:,1:]

**Czech Republic:**

In [32]:
mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis_single_group(CZ_bootstrap_single_group)
print('Czech bootstrap PBCI (Single Group):')
pbci_results.set_index('Path').T

Czech bootstrap PBCI (Single Group):


Path,Burnout Moderation Current_position_4.0,Burnout Moderation Current_position_5.0,Burnout Moderation Current_position_6.0,Satisfaction Moderation Current_position_4.0,Satisfaction Moderation Current_position_5.0,Satisfaction Moderation Current_position_6.0,Mediation,Direct Path
Salary,"[-0.012, 0.0086]","[-0.0189, 0.0037]","[-0.0191, 0.0078]","[-0.0532, 0.075]","[-0.0978, 0.0376]","[-0.0854, 0.0815]","[-0.0062, 0.0133]","[-0.0176, 0.1047]"
Effort,"[-0.001, 0.0015]","[-0.0013, 0.0015]","[-0.0022, 0.0018]","[-0.0104, -0.001] **","[-0.0123, -0.0014] **","[-0.0137, 0.0029]","[-0.0024, -0.0001] **","[0.0011, 0.0104] **"
Policy_influence,"[-0.0227, 0.03]","[-0.0189, 0.0403]","[-0.0205, 0.054]","[-0.042, 0.2708]","[-0.0388, 0.3085]","[-0.029, 0.3897] *","[-0.0399, 0.0106]","[-0.2006, 0.1087]"
Teaching_percent,"[-0.0022, 0.0009]","[-0.002, 0.0019]","[-0.0013, 0.0045]","[-0.0073, 0.0072]","[-0.0045, 0.0121]","[-0.0054, 0.0206]","[-0.001, 0.0017]","[-0.008, 0.0061]"
Research_percent,"[-0.0013, 0.0023]","[-0.0012, 0.003]","[0.0007, 0.0074] **","[-0.0079, 0.0079]","[-0.0082, 0.0112]","[-0.0099, 0.021]","[-0.0018, 0.0012]","[-0.0103, 0.0044]"
Academic_Resources,"[-0.0299, 0.0482]","[-0.0344, 0.0615]","[-0.078, 0.0691]","[-0.0067, 0.4038] *","[0.0457, 0.5229] **","[-0.1973, 0.5225]","[-0.033, 0.0404]","[0.1438, 0.5402] **"
Perceived_Autonomy,"[-0.0121, 0.0621]","[0.0059, 0.1001] **","[-0.0272, 0.0913]","[-0.347, 0.1415]","[-0.3217, 0.2012]","[-0.5305, 0.1425]","[-0.0109, 0.0586]","[0.0972, 0.548] **"


In [33]:
Current_position_col = ['Current_position_4.0', 'Current_position_5.0', 'Current_position_6.0']
path_ci, path_prob = bayesian_analysis_single_group(CZ_bayesian_single_group, Current_position_col)
print('Czech Bayesian CrI (Single Group):')
path_ci.set_index('Factor')

Czech Bayesian CrI (Single Group):


,Moderation - Group 1,Moderation - Group 2,Moderation - Group 3,Direct Path Moderation - Group 1,Direct Path Moderation - Group 2,Direct Path Moderation - Group 3,Mediation,Direct Path
Factor,,,,,,,,
Salary,"[-0.0138, 0.0077]","[-0.0201, 0.0024]","[-0.0211, 0.0068]","[-0.0422, 0.0505]","[-0.0890, 0.0147]","[-0.0717, 0.0571]","[-0.0054, 0.0150]","[0.0068, 0.0976] **"
Effort,"[-0.0011, 0.0011]","[-0.0014, 0.0009]","[-0.0024, 0.0010]","[-0.0099, -0.0006] **","[-0.0116, -0.0013] **","[-0.0127, 0.0026]","[-0.0020, 0.0001] *","[0.0009, 0.0098] **"
Teaching,"[-0.0025, 0.0006]","[-0.0023, 0.0014]","[-0.0017, 0.0040]","[-0.0065, 0.0057]","[-0.0036, 0.0113]","[-0.0066, 0.0177]","[-0.0007, 0.0019]","[-0.0065, 0.0055]"
Research,"[-0.0016, 0.0020]","[-0.0016, 0.0027]","[-0.0000, 0.0074] *","[-0.0082, 0.0087]","[-0.0072, 0.0120]","[-0.0111, 0.0202]","[-0.0016, 0.0016]","[-0.0111, 0.0044]"
Policy,"[-0.0233, 0.0283]","[-0.0195, 0.0384]","[-0.0206, 0.0543]","[-0.0221, 0.2522]","[-0.0089, 0.2813] *","[0.0052, 0.3781] **","[-0.0386, 0.0118]","[-0.1730, 0.0797]"
Acad Resources,"[-0.0365, 0.0387]","[-0.0392, 0.0487]","[-0.0769, 0.0561]","[0.0328, 0.3956] **","[0.0757, 0.5010] **","[-0.1413, 0.4921]","[-0.0252, 0.0441]","[0.1645, 0.4922] **"
Autonomy,"[-0.0087, 0.0638]","[0.0087, 0.0945] **","[-0.0294, 0.0821]","[-0.2865, 0.0418]","[-0.2624, 0.1190]","[-0.4533, 0.0739]","[-0.0089, 0.0544]","[0.1810, 0.4898] **"


In [34]:
path_prob.set_index('Factor')

,Moderation - Group 1,Moderation - Group 2,Moderation - Group 3,Direct Path Moderation - Group 1,Direct Path Moderation - Group 2,Direct Path Moderation - Group 3,Mediation,Direct Path
Factor,,,,,,,,
Salary,0.2825,0.0760,0.1625,0.5310,0.0800,0.4075,0.8140,0.9855
Effort,0.4670,0.3465,0.2370,0.0120,0.0120,0.0920,0.0390,0.9910
Teaching,0.1440,0.3355,0.7675,0.4535,0.8470,0.8355,0.8140,0.4225
Research,0.5770,0.7120,0.9725,0.5250,0.6715,0.7275,0.4655,0.2100
Policy,0.5680,0.7330,0.7755,0.9470,0.9630,0.9775,0.1535,0.2155
Acad Resources,0.5490,0.5740,0.3805,0.9940,0.9980,0.8450,0.6935,1.0000
Autonomy,0.9100,0.9910,0.8230,0.0975,0.2365,0.0760,0.9185,1.0000


**Austria:**

In [35]:
mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis_single_group(AT_bootstrap_single_group)
print('Austrian bootstrap PBCI (Single Group):')
pbci_results.set_index('Path').T

Austrian bootstrap PBCI (Single Group):


Path,Burnout Moderation Current_position_3.0,Burnout Moderation Current_position_4.0,Burnout Moderation Current_position_5.0,Burnout Moderation Current_position_6.0,Satisfaction Moderation Current_position_3.0,Satisfaction Moderation Current_position_4.0,Satisfaction Moderation Current_position_5.0,Satisfaction Moderation Current_position_6.0,Mediation,Direct Path
Salary,"[-0.0025, 0.0047]","[-0.0069, 0.0057]","[-0.0067, 0.0011]","[-0.0064, 0.0001] *","[-0.0086, 0.0353]","[-0.0684, 0.0425]","[-0.0245, 0.0486]","[-0.0136, 0.0184]","[-0.0003, 0.0055]","[-0.0164, 0.0128]"
Effort,"[-0.0011, 0.0007]","[-0.001, 0.0021]","[-0.0012, 0.0015]","[-0.0005, 0.0012]","[-0.0028, 0.0088]","[-0.0113, 0.016]","[-0.0174, 0.0016]","[0.0006, 0.0088] **","[-0.001, 0.0002]","[-0.0074, -0.0011] **"
Policy_influence,"[-0.0146, 0.0299]","[-0.0444, 0.0314]","[-0.0239, 0.0252]","[-0.0346, 0.0049]","[-0.2623, 0.1259]","[-0.2217, 0.5314]","[-0.3064, 0.4224]","[-0.17, 0.0856]","[-0.0076, 0.0255]","[-0.025, 0.184]"
Teaching_percent,"[-0.0013, 0.0013]","[-0.0005, 0.0043]","[-0.0042, 0.0004]","[-0.001, 0.0012]","[-0.0107, 0.0096]","[-0.025, 0.0094]","[-0.0144, 0.0218]","[-0.0148, 0.0008] *","[-0.0008, 0.001]","[-0.0039, 0.0092]"
Research_percent,"[-0.0006, 0.0014]","[-0.0013, 0.0023]","[-0.0006, 0.0038]","[-0.0003, 0.0026]","[-0.007, 0.0078]","[-0.0337, 0.0123]","[-0.015, 0.033]","[-0.0156, -0.0005] **","[-0.0007, 0.0007]","[-0.0006, 0.0081] *"
Academic_Resources,"[-0.0111, 0.0492]","[-0.0149, 0.0899]","[-0.0613, 0.0269]","[-0.018, 0.0376]","[-0.2884, 0.0807]","[-0.4587, 0.6438]","[-0.0044, 0.9724] *","[-0.1127, 0.2149]","[-0.0151, 0.0228]","[0.1927, 0.4389] **"
Perceived_Autonomy,"[-0.0472, 0.0055]","[-0.0412, 0.0277]","[-0.0524, 0.0144]","[-0.007, 0.0369]","[-0.256, 0.1216]","[-0.3529, 0.4977]","[-0.7688, 0.133]","[-0.0775, 0.2221]","[0.0047, 0.0603] **","[0.268, 0.4806] **"


In [36]:
Current_position_col = ['Current_position_3.0', 'Current_position_4.0', 'Current_position_5.0', 'Current_position_6.0']
path_ci, path_prob = bayesian_analysis_single_group(AT_bayesian_single_group, Current_position_col)
print('Austrian Bayesian CrI (Single Group):')
path_ci.set_index('Factor')

Austrian Bayesian CrI (Single Group):


,Moderation - Group 1,Moderation - Group 2,Moderation - Group 3,Moderation - Group 4,Direct Path Moderation - Group 1,Direct Path Moderation - Group 2,Direct Path Moderation - Group 3,Direct Path Moderation - Group 4,Mediation,Direct Path
Factor,,,,,,,,,,
Salary,"[-0.0019, 0.0043]","[-0.0073, 0.0031]","[-0.0054, 0.0024]","[-0.0053, 0.0002] *","[-0.0097, 0.0360]","[-0.0446, 0.0349]","[-0.0199, 0.0425]","[-0.0190, 0.0175]","[-0.0003, 0.0045]","[-0.0148, 0.0193]"
Effort,"[-0.0011, 0.0006]","[-0.0013, 0.0012]","[-0.0009, 0.0018]","[-0.0004, 0.0011]","[-0.0022, 0.0084]","[-0.0066, 0.0122]","[-0.0165, 0.0032]","[0.0005, 0.0093] **","[-0.0009, 0.0001]","[-0.0078, -0.0009] **"
Teaching,"[-0.0013, 0.0012]","[-0.0002, 0.0040]","[-0.0038, 0.0011]","[-0.0009, 0.0011]","[-0.0112, 0.0084]","[-0.0252, 0.0006] *","[-0.0166, 0.0180]","[-0.0165, 0.0004] *","[-0.0008, 0.0009]","[-0.0038, 0.0104]"
Research,"[-0.0005, 0.0013]","[-0.0017, 0.0023]","[-0.0010, 0.0045]","[-0.0001, 0.0025]","[-0.0072, 0.0070]","[-0.0248, 0.0065]","[-0.0171, 0.0247]","[-0.0164, 0.0008] *","[-0.0007, 0.0005]","[-0.0009, 0.0088]"
Policy,"[-0.0131, 0.0287]","[-0.0458, 0.0240]","[-0.0347, 0.0332]","[-0.0343, 0.0041]","[-0.2263, 0.0984]","[-0.0690, 0.4296]","[-0.2217, 0.2925]","[-0.1810, 0.1023]","[-0.0064, 0.0229]","[-0.0362, 0.1896]"
Acad Resources,"[-0.0108, 0.0413]","[-0.0220, 0.0693]","[-0.0602, 0.0432]","[-0.0168, 0.0346]","[-0.2985, 0.0923]","[-0.2476, 0.4275]","[0.0019, 0.8116] **","[-0.1183, 0.2332]","[-0.0115, 0.0245]","[0.1815, 0.4496] **"
Autonomy,"[-0.0440, 0.0036]","[-0.0452, 0.0259]","[-0.0552, 0.0296]","[-0.0066, 0.0353]","[-0.2490, 0.1090]","[-0.2314, 0.3151]","[-0.5871, 0.0621]","[-0.0809, 0.2315]","[0.0031, 0.0592] **","[0.2595, 0.4907] **"


In [37]:
path_prob

,Factor,Moderation - Group 1,Moderation - Group 2,Moderation - Group 3,Moderation - Group 4,Direct Path Moderation - Group 1,Direct Path Moderation - Group 2,Direct Path Moderation - Group 3,Direct Path Moderation - Group 4,Mediation,Direct Path
0,Salary,0.7365,0.2675,0.2545,0.0425,0.8665,0.3985,0.7650,0.4835,0.9385,0.5795
1,Effort,0.2735,0.4430,0.7005,0.7655,0.8695,0.7070,0.0805,0.9830,0.1165,0.0130
2,Teaching,0.5230,0.9480,0.2065,0.5770,0.4000,0.0325,0.5225,0.0315,0.5025,0.8145
3,Research,0.7795,0.6440,0.8605,0.9465,0.4865,0.1210,0.6400,0.0375,0.4290,0.9445
4,Policy,0.7385,0.3040,0.4910,0.0920,0.2215,0.9125,0.6175,0.2945,0.8105,0.9085
5,Acad Resources,0.8325,0.8225,0.3960,0.7235,0.1335,0.6905,0.9755,0.7300,0.7150,1.0000
6,Autonomy,0.0745,0.2920,0.2800,0.8705,0.2495,0.6245,0.0545,0.8235,0.9840,1.0000
